## Haar Cascades

Based on the popular (Viola-Jones)[https://link.springer.com/article/10.1023/B:VISI.0000013087.49260.fb] framework for object detection. They proposed using Haar features (two-valued,  rectangular masks), efficiently computed using "integral" images (where each pixel value is the sum of all the pixel values in the rectangular zone where that pixel is the bottom-right corner). Something like a 2D, discrete CDF in probability theory. They also proposed using a "cascade" of weak classifiers to improve performance.

OpenCV offers an implementation of this under the "CascadeClassifier" class, along with pretrained classifiers for face detection. 

In [37]:
import cv2

def detect_faces_haar(image_path, output_path): 
    # Path to preset face detection cascade file that come with OpenCV out of the box. Some other cascade files can be found at venv/Lib/site-packages/cv2/data
    face_detection_cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    face_cascade = cv2.CascadeClassifier(face_detection_cascade_file)     

    image = cv2.imread(image_path)
    cv2.imshow("Base image", image)
    faces_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Parameters for detectMultiScale taken from https://stackoverflow.com/questions/20801015/recommended-values-for-opencv-detectmultiscale-parameters 
    # along with some experiments on the sample image
    faces = face_cascade.detectMultiScale(faces_gray, scaleFactor = 1.2, minNeighbors = 4)      # these values can be changed as needed 

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imwrite(output_path, image)
    cv2.imshow("Detected faces (Haar Cascade)", image)
    print("Close windows or press key to exit.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [38]:
detect_faces_haar("samples/face_detection/solvay_color.jpg", "samples/face_detection/solvay_detected_haar.jpg")

Close windows or press key to exit.
